In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


In [5]:
sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
output = model(input_ids)


In [6]:

print("Input IDs:", input_ids)
print("")
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])

Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


### Task: 
Apply the tokenization manually on the two sentences used in section 2 (“I’ve been waiting for a HuggingFace course my whole life.” and “I hate this so much!”). Pass them through the model and check that you get the same logits as in section 2. Now batch them together using the padding token, then create the proper attention mask. Check that you obtain the same results when going through the model!

In [12]:
sequences = ["I’ve been waiting for a HuggingFace course my whole life.", 
             "I hate this so much!"]

In [16]:
# este paso no es necesario
tokens = [ tokenizer.tokenize(x) for x in sequences ]
ids = [tokenizer.convert_tokens_to_ids(x) for x in tokens]


In [29]:
inputs = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1045,  1521,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [30]:
output = model(**inputs)
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5979,  1.6390],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
predictions

tensor([[3.7801e-02, 9.6220e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)

In [39]:
for seq, pred, in zip(sequences, predictions):
    print(seq)
    for pr, tag in zip(pred, model.config.id2label):
        print(pr.item(), model.config.id2label[tag])
    print('\n')

I’ve been waiting for a HuggingFace course my whole life.
0.037801433354616165 NEGATIVE
0.9621986150741577 POSITIVE


I hate this so much!
0.9994558691978455 NEGATIVE
0.0005441844696179032 POSITIVE


